## Use Langchain and Open AI to interact with a Databricks Tables

In [0]:

#import necessary libraries 

import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain import OpenAI



os.environ["OPENAI_API_KEY"] =" "
 
#Connect to database and create agent

db = SQLDatabase.from_databricks(catalog="dev_db", schema="core")
llm = OpenAI(temperature=.7)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.


In [0]:
#Ask Questions in Simple english and see how the agent converts it to sql to respond

agent.run("How many companies do we have with an ipo status")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: active_companies_information
Thought:

DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.


 I should query the schema of the active_companies_information table
Action: sql_db_schema
Action Input: "active_companies_information"

DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.
DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.



Observation: 
CREATE TABLE active_companies_information (
	org_uuid STRING, 
	name STRING, 
	org_type STRING, 
	country_code STRING, 
	homepage_url STRING, 
	status STRING, 
	description STRING, 
	investments STRING, 
	company_information STRING
) USING DELTA

/*
3 rows from active_companies_information table:
org_uuid	name	org_type	country_code	homepage_url	status	description	investments	company_information
8c57c3c2-e65b-e45b-c754-89dc9f85580a	Gaia Online	organization	USA	https://www.gaiaonline.com/	operating	Gaia Online has grown into one of the biggest forum communities in the world. Today, Gaia is the bes	IVP investment in Series C - Gaia Online	Gaia Online has grown into one of the biggest forum communities in the world. Today, Gaia is the bes
6d2fff57-32b5-f75c-9986-97539d1bf366	Viddler	organization	USA	http://www.viddler.com	operating	Viddler has a long history of innovation in online video, developing patented new approaches, and tr		Viddler has a long history of innovation in

DatabricksRetryPolicy is currently bypassed. The CommandType cannot be set.


 I should query the active_companies_information table for companies with an ipo status
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM active_companies_information WHERE status='ipo';
Observation: [(41838,)]
Thought: I now know the final answer
Final Answer: There are 41838 companies with an ipo status.

> Finished chain.
Out[2]: 'There are 41838 companies with an ipo status.'

In [0]:
#Read table into dataframe

df_companies_info = spark.read.table("dev_db.core.active_companies_information")

In [0]:
#Import necessary library

import openai
import tiktoken
from pyspark.ml.linalg import *
from pyspark.sql.types import * 
from pyspark.sql.functions import *
from langchain.embeddings.openai import OpenAIEmbeddings

In [0]:
KEY=" "

In [0]:
df_companies_info.count()

Out[6]: 2755555

In [0]:
#Create function to get number of tokens from 'company_information' column  

def num_tokens_from_string(string: str) -> int:
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [0]:
#Get tokens from 'company_information' column to see if there are values with large tokens

tokenUDF = udf(lambda x:num_tokens_from_string(x)) 
df_with_token = df_companies_info.withColumn("num_tokens", tokenUDF(col("company_information")))
df_with_token = df_with_token.withColumn("num_tokens",col("num_tokens").cast("int"))
df_with_token.select(col("company_information"),col("num_tokens")).show()

+--------------------+----------+
| company_information|num_tokens|
+--------------------+----------+
|Gaia Online has g...|       118|
|Viddler has a lon...|       197|
|Music Intelligenc...|       173|
|Ampersand is a mi...|        64|
|Akampus is a yout...|       175|
|Alibris (pronounc...|       169|
|We're your NorCal...|        72|
|AlphaClone is an ...|       156|
|Profit Point Inc....|       154|
|Info USA offers a...|       105|
|Since 2009, VENYO...|       403|
|Kenta Biotech is ...|       125|
|Munich, Germany-b...|       341|
|Stellaris is a we...|       178|
|Enterprise Divers...|       151|
|Continuity is a p...|       212|
|Tweetminster is a...|       104|
|Intelligent Clear...|       166|
|Online Project Ma...|       119|
|Reval is a SaaS-b...|       158|
+--------------------+----------+
only showing top 20 rows



In [0]:
#Create Function to extract sectors using openAI and Langchain

from langchain import PromptTemplate

def extract_company_sectors(text):
    resume_template = """
    Assuming this is a brief description of a company. Extract key information and respond with an array of sectors the company is in. Do not output anything except for the extracted information. Do not add any clarifying information.All output must be in array format below 
    []

    {text}
    """

    prompt = PromptTemplate(
        input_variables=["text"],
        template=resume_template
    )

    summary_prompt = prompt.format(text=text)
    summary = llm(summary_prompt)

    return summary.strip()

In [0]:
# Register sectors function
sectorUDF = udf(extract_company_sectors, StringType())


#Get sectors and create column called "sectors"
df_with_sectors = df_with_token.withColumn(
    "sectors", sectorUDF(df_with_token["company_information"])
)

df_with_sectors = df_with_sectors.select(col("org_uuid"),col("name"), col("sectors")).show()

df_with_sectors.show()

+--------------------+--------------------+--------------------+
|            org_uuid|                name|             sectors|
+--------------------+--------------------+--------------------+
|8c57c3c2-e65b-e45...|         Gaia Online|['Forum Community...|
|6d2fff57-32b5-f75...|             Viddler|['Online Video', ...|
|0cf4f50a-b749-04b...|Music Intelligenc...|["Digital Media",...|
|6580c555-b205-021...|Ampersand Capital...|['Healthcare', 'I...|
|5f932f59-dcec-0fa...|             Akampus|['Youth Marketing...|
|8d71fc18-7185-5f3...|             Alibris|["Books", "Music"...|
|fb70402d-641d-97b...|                KQED|["Media", "Public...|
|bbf2555d-6b35-210...|          AlphaClone|["Investment Advi...|
|e95920a2-4124-862...|        Profit Point|['Infrastructure ...|
|37814cee-be4c-3c6...|             InfoUSA|['Sales and Marke...|
|8f259024-34d5-0a2...|               Venyo|['Aviation', 'Tra...|
|646e87fd-ecf8-896...|       Kenta Biotech|["Biotechnology",...|
|4d473e5b-b6e9-146...|   

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-2147251658851979>:12
      6 df_with_sectors = df_with_token.withColumn(
      7     "sectors", sectorUDF(df_with_token["company_information"])
      8 )
     10 df_with_sectors = df_with_sectors.select(col("org_uuid"),col("name"), col("sectors")).show()
---> 12 df_with_sectors.show()

AttributeError: 'NoneType' object has no attribute 'show'